Задание 1

In [34]:
#Взято с https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt и немного переделан вывод...

#!pip install transformer
from transformers import AutoTokenizer
from collections import defaultdict

corpus = ["In a hole in the ground there lived a hobbit. Not a nasty, dirty, wet hole, filled with the ends of worms and an oozy smell, nor yet a dry, bare, sandy hole with nothing in it to sit down on or to eat: it was a hobbit-hole, and that means comfort. It had a perfectly round door like a porthole, painted green, with a shiny yellow brass knob in the exact middle. The door opened on to a tube-shaped hall like a tunnel: a very comfortable tunnel without smoke, with panelled walls, and floors tiled and carpeted, provided with polished chairs, and lots and lots of pegs for hats and coats - the hobbit was fond of visitors. The tunnel wound on and on, going fairly but not quite straight into the side of the hill - The Hill, as all the people for many miles round called it - and many little round doors opened out of it, first on one side and then on another. No going upstairs for the hobbit: bedrooms, bathrooms, cellars, pantries (lots of these), wardrobes (he had whole rooms devoted to clothes), kitchens, dining-rooms, all were on the same floor, and indeed on the same passage. The best rooms were all on the lefthand side (going in), for these were the only ones to have windows, deep-set round windows looking over his garden, and meadows beyond, sloping down to the river."]

tokenizer = AutoTokenizer.from_pretrained("gpt2") #(пре)токенизатор из GPT2

word_freqs = defaultdict(int)

for text in corpus: #тут вычисляется частотность слов в исходном тексте.
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

alphabet = [] #массив алфавита

for word in word_freqs.keys(): #в него закидываются все буквы (из входного текста)
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

vocab = ["<|endoftext|>"] + alphabet.copy() #тег конца текста и словарь закидываются в словарь.
splits = {word: [c for c in word] for word in word_freqs.keys()}

def compute_pair_freqs(splits): #происходит разделение слов на буквы...
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs


pair_freqs = compute_pair_freqs(splits)

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items(): #нахождение самой частотной пары
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

def merge_pair(a, b, splits): #здесь происходит объединение пары
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

vocab_size = 400 #400 наш максимум

while len(vocab) < vocab_size: #тут какая-то магия объединяет всё в финальный словарь
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

for elem in vocab:
  print (elem.replace('Ġ',''), sep='\n') #вывожу словарь без символа Ġ, который там, кажется, служебный.

<|endoftext|>
(
)
,
-
.
:
H
I
N
T
a
b
c
d
e
f
g
h
i
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z

th
a
nd
o
the
w
the
le
in
or
h
it
ro
and
s
on
ed
t
d
f
ll
p
wi
c
lo
ho
b
ing
om
er
und
with
of
to
ow
m
ro
he
in
g
as
ir
ar
an
ha
id
ts
oms
l
hob
hobb
hobbit
n
et
led
it
round
en
T
The
tu
el
for
es
hole
li
ot
e
me
sa
ole
ly
do
door
ke
tun
tunn
tunnel
ver
ut
is
air
lo
-
oing
sid
side
all
rooms
(
),
wer
were
ows
re
N
ty
lled
an
y
dow
down
at
was
hole
me
com
comf
comfor
comfort
had
ct
like
ell
ass
k
op
open
opened
pan
flo
floor
airs
lots
pe
ond
going
tr
ill
man
many
st
thes
these
ard
rooms
de
same
wind
windows
In
gro
ground
there
liv
lived
Not
nas
nasty
dir
dirty
wet
fi
filled
end
ends
wor
worm
worms
oo
ooz
oozy
sme
smell
nor
yet
dr
dry
bar
bare
sand
sandy
no
noth
nothing
sit
or
eat
th
that
mean
means
I
It
per
perf
perfe
perfect
perfectly
por
porth
porthole
pa
pain
paint
painted
gre
green
sh
shin
shiny
yell
yellow
br
brass
kn
kno
knob
ex
exa
exact
mid
midd
middle
tub
tube
sh
sha
shap
shaped
hall
ver
very
c

Задание 2

In [1]:
#сам алгоритм вычислений взят на Хабре, я дописал сюда всё связанное с выводом матрицы.

import pandas as pd

str_1 = 'программирование'
str_2 = 'лингвистика'

check = 0
matrix = [] #тут будет матрица со всеми цифрами

n, m = len(str_1), len(str_2) #строки на вход
if n > m: #m должно быть длиннее n, меняем порядок
	str_1, str_2 = str_2, str_1
	n, m = m, n
	check = 1


def levenstein(str_1, str_2):

    current_row = range (n + 1) #нулевой ряд от 0 до n+1
    for i in range (1, m + 1): #алгоритм в цикле
        previous_row, current_row = current_row, [i] + [0] * n #нынешний ряд -> предыдущий,
        matrix.append(list(previous_row)) #запись матрицы
        for j in range (1, n + 1): #двигаемся по строчке в цикле
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1] #у нас три значения: D(i,j-1), D(i-1,j) и D(i-1,j-1) + m (S1[i], S2[j])
            if str_1[j - 1] != str_2[i - 1]: #если буквы разные
                change += 1 #записываем единицу в m
            current_row[j] = min(add, delete, change) #находим из них минимальное и записываем его в таблицу
    matrix.append (current_row) #запись последнего ряда в матрицу

    return current_row[n]

placeholder = [' '] #ниже строки для того, чтобы дописать слова в таблицу с числами.

def tables (str_1, str_2, df):
	df.columns = placeholder+[*str_1]
	df.index = placeholder+[*str_2]
	return 0

levenstein (str_1, str_2) #инициализация функции
df = pd.DataFrame (matrix)
tables (str_1, str_2, df) #дописываем

print ('результат: ', levenstein (str_1, str_2), '\n', '\n')
#print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in matrix]))
print (df, '\n\n')

#а для levenshtein и einstein там 5 будет

результат:  13 
 

        л   и   н   г   в   и   с   т   и   к   а
    0   1   2   3   4   5   6   7   8   9  10  11
п   1   1   2   3   4   5   6   7   8   9  10  11
р   2   2   2   3   4   5   6   7   8   9  10  11
о   3   3   3   3   4   5   6   7   8   9  10  11
г   4   4   4   4   3   4   5   6   7   8   9  10
р   5   5   5   5   4   4   5   6   7   8   9  10
а   6   6   6   6   5   5   5   6   7   8   9   9
м   7   7   7   7   6   6   6   6   7   8   9  10
м   8   8   8   8   7   7   7   7   7   8   9  10
и   9   9   8   9   8   8   7   8   8   7   8   9
р  10  10   9   9   9   9   8   8   9   8   8   9
о  11  11  10  10  10  10   9   9   9   9   9   9
в  12  12  11  11  11  10  10  10  10  10  10  10
а  13  13  12  12  12  11  11  11  11  11  11  10
н  14  14  13  12  13  12  12  12  12  12  12  11
и  15  15  14  13  13  13  12  13  13  12  13  12
е  16  16  15  14  14  14  13  13  14  13  13  13 




Задание 3

In [2]:
import re

txt = "why_n-ot@why.ru.jp"
x = re.search("[\w-]+@[\w+-]+\\.[\w.]+", txt) #может понимать разные домены первого уровня

if x:
  print("Works")
else:
  print("Doesn't work")

Works


In [3]:
import re

txt = "/mnt/c/Windows/System32/drivers/etc/hosts.txt"
x = re.search("/|^$\w:|^$(?:[\\/]\w+\.\w+)", txt)

if x:
  print("Works")
else:
  print("Doesn't work")

Works


In [78]:
import re

txt = """ #[[Time: Обычно"обычно:#frequentative_adverbs_adj:FREQUENTATIVE"] [Experiencer_Metaphoric: бюджет"бюджет:бюджет:BUDGET"] [[ко"к:#preposition:PREPOSITION"] [OrderInTimeAndSpace: второму"второй:TWO_ORDINAL"] Object_Situation: чтению "чтение:READING_OF_THE_DRAFT_LAW"] Predicate: готовится"готовить:готовить:PREPAREDNESS" [[DegreeApproximative: непосредственно"непосредственный:DIRECT_OBLIQUE"] [в"в_Prepositional:#preposition:PREPOSITION"] Locative: Думе"дума:дума:DUMA"]#: [[Agent: депутаты"депутат:депутат:DEPUTY"] Specification_Clause: корректируют"корректировать:корректировать:TO_CORRECT" [[Agent: правительственные"правительство:правительство:GOVERNMENT"] Object_Situation: планы"план:план:SCHEDULE_FOR_ACTIVITY"]]]. """

occ = re.findall('\[(\w+): ', txt) #сем. роль
occ1 = re.findall(': (\w+)\"', txt) #слово
occ2 = re.findall('\"(\w+):', txt) #лемма
occ3 = re.findall(':(#\w+|\w+):', txt) #лексический класс
occ4 = re.findall(':(\w+)\"[\] ]', txt) #семантический класс


print(occ, occ1, occ2, occ3, occ4, sep='\n\n')

['Time', 'Experiencer_Metaphoric', 'OrderInTimeAndSpace', 'DegreeApproximative', 'Agent', 'Agent']

['Обычно', 'бюджет', 'второму', 'готовится', 'непосредственно', 'Думе', 'депутаты', 'корректируют', 'правительственные', 'планы']

['обычно', 'бюджет', 'к', 'второй', 'чтение', 'готовить', 'непосредственный', 'в_Prepositional', 'дума', 'депутат', 'корректировать', 'правительство', 'план']

['#frequentative_adverbs_adj', 'бюджет', '#preposition', 'готовить', '#preposition', 'дума', 'депутат', 'корректировать', 'правительство', 'план']

['FREQUENTATIVE', 'BUDGET', 'PREPOSITION', 'TWO_ORDINAL', 'READING_OF_THE_DRAFT_LAW', 'PREPAREDNESS', 'DIRECT_OBLIQUE', 'PREPOSITION', 'DUMA', 'DEPUTY', 'TO_CORRECT', 'GOVERNMENT', 'SCHEDULE_FOR_ACTIVITY']
